# Python地理信息向量数据处理介绍

In [ ]:
%matplotlib inline

import pandas as pd
import geopandas

pd.options.display.max_rows = 10

## 导入地理信息数据

地理信息数据大概率来自一些特定的 GIS 格式文件或者数据库, 比如 ESRI shapefiles, GeoJSON, geopackage, PostGIS (PostgreSQL) 数据库等等。

我们可以使用 GeoPandas 的 `geopandas.read_file` 函数来读取这些格式的文件(底层依赖 `fiona` , 一个对 GDAL/OGR 的封装)。

For example, let's start by reading a shapefile with all the countries of the world (adapted from http://www.naturalearthdata.com/downloads/110m-cultural-vectors/110m-admin-0-countries/, zip file is available in the `/data` directory), and inspect the data:

例如，我们从读取并查看一个包含全部国家的 shapefile 文件开始(引自：http://www.naturalearthdata.com/downloads/110m-cultural-vectors/110m-admin-0-countries/，zip 文件已经在`/data`文件夹下)。

In [ ]:
countries = geopandas.read_file("zip://./data/ne_110m_admin_0_countries.zip")
# 如果 zip 文件已经解压过:
# countries = geopandas.read_file("data/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")

In [ ]:
countries.head()

In [ ]:
countries.plot()

我们可以观察到:

- 使用 `.head()` 我们可以看到 dataset 的第一行, 就像在 Pandas 中一样
- 有一个 'geometry' 列，每个国家都用多边形描述出来
- 可以使用 `.plot()` 方法快速获取数据的基本可视化

## 什么是 GeoDataFrame (地理数据框架)?

我们使用 GeoPandas 读取地理信息数据, GeoPandas 就会返回 `GeoDataFrame`:

In [ ]:
type(countries)

一个 GeoDataFrame 包含表格和地理信息数据集。

* 它有一个 **'几何图形' 列** 用来存储几何信息 (或者 GeoJSON 中的特征).
* 其他列 **属性** (或者 GeoJSON 中的特征) 用来描述这些几何图形

这样的 `GeoDataFrame` 就像 pandas 里的 `DataFrame`, 但是附加了一些处理地理信息数据的功能:

* `.geometry` 属性总是返回几何信息 (返回 GeoSeries). 改列的列名也不一定非得是 'geometry', 但是这一列总会被作为 `.geometry` 属性处理.
* 有一些处理地理数据的方法 (面积, 距离, 缓冲区, 交集, ...), 后面的教程中会讲到

In [ ]:
countries.geometry

In [ ]:
type(countries.geometry)

In [ ]:
countries.geometry.area

**它仍然是 DataFrame**, 所以所有的 pandas 功能都可以用在地理信息数据集上, 并对各属性和几何图形数据进行处理。

比如, 我们可以计算各个国家的平均人口 (通过在 'pop_est' 列上调用 `mean` 方法):

In [ ]:
countries['pop_est'].mean()

或者,我们可以通过表达式使用布尔过滤器选择 dataframe 的子集：

In [ ]:
africa = countries[countries['continent'] == 'Africa']

In [ ]:
africa.plot()

---

接下来的教程会假设你已经有一些 pandas 基础, 但是也会对不熟悉的地方给一些建议.   
如果想要更深入的了解 pandas，可以看看以下资源:

- Pandas docs: https://pandas.pydata.org/pandas-docs/stable/10min.html
- Other tutorials: chapter from pandas in https://jakevdp.github.io/PythonDataScienceHandbook/, https://github.com/jorisvandenbossche/pandas-tutorial, https://github.com/TomAugspurger/pandas-head-to-tail, ...

<div class="alert alert-info" style="font-size:120%">
<b>提醒</b>: <br>

<ul>
  <li>`GeoDataFrame` 允许使用地理计算处理典型表格数据</li>
  <li>`GeoDataFrame` (或者 *特征集合*) 由以下组成:
   <ul>
    <li>**Geometries** or **features**: 空间对象</li>
    <li>**Attributes** or **properties**: 关于空间对象包含信息的列</li>
   </ul>
  </li>
</ul>
</div>

## 几何图形: 点、线、面

空间 **向量** 数据有三种不同的基本类型组成:

* **Point** 数据: 描述空间中的一个点.
* **Line** 数据 ("LineString"): 使用若干个点描述一条线.
* **Polygon** 数据: 使用多边形描述一个填充平面.

它们也可以组成符合几何图形 (扩展知识： https://shapely.readthedocs.io/en/stable/manual.html#geometric-objects).

可以看到, 个体空间对象就是多边形:

In [ ]:
print(countries.geometry[2])

我们来导入一些不同类型几何对象的数据集.

世界城市数据集 (引用： http://www.naturalearthdata.com/downloads/110m-cultural-vectors/110m-populated-places/, zip 文件已保存在 `/data` 文件夹下), 由点数据组成:

In [ ]:
cities = geopandas.read_file("zip://./data/ne_110m_populated_places.zip")

In [ ]:
print(cities.geometry[0])

还有一个世界河流数据集 (引用： http://www.naturalearthdata.com/downloads/50m-physical-vectors/50m-rivers-lake-centerlines/, zip 文件已保存在 `/data` 文件夹下) ，由线数据组成:

In [ ]:
rivers = geopandas.read_file("zip://./data/ne_50m_rivers_lake_centerlines.zip")

In [ ]:
print(rivers.geometry[0])

###  `shapely` 库

几何对象来自 [`shapely`](https://shapely.readthedocs.io/en/stable/) 库

In [ ]:
type(countries.geometry[0])

我们自己来造一个几何对象:

In [ ]:
from shapely.geometry import Point, Polygon, LineString

In [ ]:
p = Point(1, 1)

In [ ]:
print(p)

In [ ]:
polygon = Polygon([(1, 1), (2,2), (2, 1)])

<div class="alert alert-info" style="font-size:120%">
<b>提醒</b>: <br><br>

单个几何图形由 `shapely` 对象体现:

<ul>
  <li>如果你访问 GeoDataFrame 里的单个几何对象, 你会得到一个 `shapely` 几何对象</li>
  <li>这些对象和 geopandas 对象具有类似的方法 (GeoDataFrame/GeoSeries). 例如:
   <ul>
    <li>`single_shapely_object.distance(other_point)` -> 两点之间的距离</li>
    <li>`geodataframe.distance(other_point)` -> geodataframe 中每个点到 `other_point` 的距离</li>
   </ul>
  </li>
</ul>
</div>

## 坐标参考系

**坐标参考系 coordinate reference system (CRS)** 表明一个二维坐标系下的几何对象如何关联到地球上实际的地点.

深入理解请看：
https://docs.qgis.org/2.8/en/docs/gentle_gis_introduction/coordinate_reference_systems.html

GeoDataFrame 或者 GeoSeries 都有 `.crs` 属性， 用来保存坐标参考系的描述:

In [ ]:
countries.crs

`countries` dataframe 表明它在使用 EPSG 4326 / WGS84 经纬参考系, 最常用的坐标系.  
像你在下图中看到的这样，它用经纬度表示坐标:

In [ ]:
countries.plot()

`.crs` 属性是个字典。在这个案例里，它只包含 EPSG 码，但是它也可以包含完整的『proj4』字符串（字典格式）。

在底层, GeoPandas 使用 `pyproj` / `proj4` 库来处理重投影。

更多信息，请看 http://geopandas.readthedocs.io/en/latest/projections.html

---

你可能会有很多理由需要转换数据集的坐标系, 比如:

- 不同的源使用不同的坐标参考系 -> 需要转换成相同的坐标参考系
- 基于距离的操作 -> 使用一个以米为单位的坐标参考系 (而不是角度)
- 使用一个固定的坐标参考系绘图 (比如要保持面积)

可以使用 `to_crs` 函数来转换 GeoDataFrame 的坐标参考系. 

例如, 我们把国家数据集转换成世界墨卡托投影 (http://epsg.io/3395):

In [ ]:
# 删除南极, 墨卡托投影处理不了两极
countries = countries[(countries['name'] != "Antarctica")]

In [ ]:
countries_mercator = countries.to_crs(epsg=3395)  # or .to_crs({'init': 'epsg:3395'})

In [ ]:
countries_mercator.plot()

注意 x 和 y 尺度不同

## 讲不同的层绘制到一起

In [ ]:
ax = countries.plot(edgecolor='k', facecolor='none', figsize=(15, 10))
rivers.plot(ax=ax)
cities.plot(ax=ax, color='red')
ax.set(xlim=(-20, 60), ylim=(-40, 40))

更多地理数据可视化详情，请看笔记 [04-more-on-visualization.ipynb](04-more-on-visualization.ipynb)。

## 更多的 GeoDataFrames 导入和创建只是

### 关于 `fiona`

在底层, GeoPandas 使用 [Fiona library](http://toblerity.org/fiona/) (GDAL/OGR 的 Python 封装) 来读写数据. 对于大部分常用场景，GeoPandas 提供了更加用户友好的包装. 如果你想进行更细节的控制, 那么可以这样用 fiona 读取数据:


In [ ]:
import fiona
from shapely.geometry import shape

with fiona.drivers():
    with fiona.open("data/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp") as collection:
        for feature in collection:
            # ... do something with geometry
            geom = shape(feature['geometry'])
            # ... do something with properties
            print(feature['properties']['name'])

### 手工构建 GeoDataFrame

In [ ]:
geopandas.GeoDataFrame({
    'geometry': [Point(1, 1), Point(2, 2)],
    'attribute1': [1, 2],
    'attribute2': [0.1, 0.2]})

### Creating a GeoDataFrame from an existing dataframe

For example, if you have lat/lon coordinates in two columns:

In [ ]:
df = pd.DataFrame(
    {'City': ['Buenos Aires', 'Brasilia', 'Santiago', 'Bogota', 'Caracas'],
     'Country': ['Argentina', 'Brazil', 'Chile', 'Colombia', 'Venezuela'],
     'Latitude': [-34.58, -15.78, -33.45, 4.60, 10.48],
     'Longitude': [-58.66, -47.91, -70.66, -74.08, -66.86]})

In [ ]:
df['Coordinates']  = list(zip(df.Longitude, df.Latitude))

In [ ]:
df['Coordinates'] = df['Coordinates'].apply(Point)

In [ ]:
gdf = geopandas.GeoDataFrame(df, geometry='Coordinates')

In [ ]:
gdf

See http://geopandas.readthedocs.io/en/latest/gallery/create_geopandas_from_pandas.html#sphx-glr-gallery-create-geopandas-from-pandas-py for full example